In [16]:
import json

# Helper function to convert latitude and longitude to cartesian coordinates
def convert_to_grid_coords(lat, lng, lat_min, lat_max, lng_min, lng_max, grid_size=1000, depot_position=(500, 500)):
    # Normalizing latitude and longitude
    lat_norm = (lat - lat_min) / (lat_max - lat_min)
    lng_norm = (lng - lng_min) / (lng_max - lng_min)

    # Scaling to the grid size
    x = int(lng_norm * grid_size)
    y = int(lat_norm * grid_size)

    # Translating so that depot is at the specified position
    x += (depot_position[0] - int(grid_size / 2))
    y += (depot_position[1] - int(grid_size / 2))

    return x, y

# Path to the JSON file
file_path_json = '/Users/cstenico/Documents/shire/tcc-mba/loggibud/data/025/vrppd-instances-1.0/train/df-0/cvrp-0-df-0.json'


# Loading the JSON data
with open(file_path_json, 'r') as file:
    json_data = json.load(file)

# Extracting depot data
depot = json_data['depot']
# Finding the min and max latitude and longitude values from the JSON data
latitudes = [demand['point']['lat'] for demand in json_data['demands']] + [depot['lat']]
longitudes = [demand['point']['lng'] for demand in json_data['demands']] + [depot['lng']]
lat_min, lat_max = min(latitudes), max(latitudes)
lng_min, lng_max = min(longitudes), max(longitudes)

# Converting the depot and demands to VRP format with the new coordinate system
depot_coords = convert_to_grid_coords(depot['lat'], depot['lng'], lat_min, lat_max, lng_min, lng_max)
demands = [f"1\t0"]  # Demand at depot is zero

# Preparing nodes with the new coordinate system
nodes = [f"1\t{depot_coords[0]}\t{depot_coords[1]}"]  # Depot as the first node
for i, demand in enumerate(json_data['demands'], start=2):
    node_coords = convert_to_grid_coords(demand['point']['lat'], demand['point']['lng'], lat_min, lat_max, lng_min, lng_max)
    nodes.append(f"{i}\t{node_coords[0]}\t{node_coords[1]}")
    demands.append(f"{i}\t{demand['size']}")

# Constructing the VRP file content
vrp_content = f"NAME : {json_data['name']}\n"
vrp_content += "TYPE : CVRP\n"
vrp_content += f"DIMENSION : {len(nodes)}\n"
vrp_content += "EDGE_WEIGHT_TYPE : EUC_2D\n"
vrp_content += f"CAPACITY : {json_data['vehicle_capacity']}\n"
vrp_content += "NODE_COORD_SECTION\n"
vrp_content += '\n'.join(nodes)
vrp_content += "\nDEMAND_SECTION\n"
vrp_content += '\n'.join(demands)
vrp_content += "\nDEPOT_SECTION\n1\n-1\nEOF"


In [17]:
vrp_content

'NAME : cvrp-0-df-0\nTYPE : CVRP\nDIMENSION : 1285\nEDGE_WEIGHT_TYPE : EUC_2D\nCAPACITY : 180\nNODE_COORD_SECTION\n1\t349\t711\n2\t321\t732\n3\t327\t733\n4\t328\t726\n5\t318\t752\n6\t318\t724\n7\t324\t731\n8\t326\t730\n9\t333\t737\n10\t330\t727\n11\t330\t728\n12\t333\t732\n13\t326\t741\n14\t326\t742\n15\t324\t729\n16\t324\t729\n17\t325\t735\n18\t326\t731\n19\t333\t734\n20\t321\t727\n21\t329\t742\n22\t326\t734\n23\t329\t731\n24\t325\t729\n25\t333\t733\n26\t331\t737\n27\t330\t741\n28\t325\t732\n29\t326\t742\n30\t333\t732\n31\t325\t735\n32\t277\t978\n33\t259\t879\n34\t572\t817\n35\t571\t816\n36\t561\t815\n37\t571\t816\n38\t575\t816\n39\t560\t816\n40\t571\t815\n41\t553\t814\n42\t563\t811\n43\t570\t807\n44\t563\t817\n45\t559\t802\n46\t557\t817\n47\t570\t815\n48\t568\t811\n49\t573\t812\n50\t565\t816\n51\t577\t818\n52\t564\t808\n53\t560\t816\n54\t567\t815\n55\t566\t810\n56\t565\t812\n57\t559\t815\n58\t556\t809\n59\t573\t814\n60\t572\t815\n61\t571\t813\n62\t656\t1000\n63\t562\t812\n64\t555\t81

In [18]:
output_file_path = 'instances/DF/DF.vrp'

# Saving the VRP content to a file
with open(output_file_path, 'w') as file:
    file.write(vrp_content)

In [20]:

nb_customers=len(json_data['demands']) - 1,
depot_position='R',
customer_position='R',
nb_customer_cluster=None,
demand_type='inter',
demand_min=min(demand['size'] for demand in json_data['demands']),
demand_max=max(demand['size'] for demand in json_data['demands']),
capacity=json_data['vehicle_capacity'],
grid_size=1000

In [23]:
nb_customers

(1283,)